<a href="https://colab.research.google.com/github/liuezhong/homework/blob/master/capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightgbm xgboost catboost category-encoders sklearn pandas==1.1.5

In [ ]:
!git clone https://github.com/liuezhong/homework.git

In [3]:
import pandas as pd
import numpy as np

train = pd.read_csv('./homework/two/dataset/train_final.csv', engine='python')
test = pd.read_csv('./homework/two/dataset/test_final.csv', engine='python')

In [ ]:
train.columns.to_list()

In [ ]:
train["continuous_loan_amnt"].describe(percentiles=np.arange(0, 1,0.05))

In [ ]:
train["continuous_annual_inc"].describe(percentiles=np.arange(0, 1,0.05))

In [ ]:
pd.isnull(train['continuous_last_fico_range_high']).sum()

In [ ]:
train[['loan_status', 'continuous_delinq_2yrs']].groupby('loan_status').describe(percentiles=np.arange(0, 1,0.05))

In [ ]:
train[['loan_status', 'continuous_annual_inc']].groupby('loan_status').describe(percentiles=np.arange(0, 1,0.05))

In [ ]:
train[['loan_status', 'continuous_int_rate']].groupby('loan_status').describe(percentiles=np.arange(0, 1,0.05))

In [ ]:
train['discrete_is_loan_10000'] = (train['continuous_loan_amnt'] ==10000).astype(np.int64)
test['discrete_is_loan_10000'] = (train['continuous_loan_amnt'] ==10000).astype(np.int64)

In [5]:
train['discrete_rate_greater_917'] = (train['continuous_int_rate'] > 9.17).astype(np.int64)
test['discrete_rate_greater_917'] = (train['continuous_int_rate'] > 9.17).astype(np.int64)

In [ ]:
train['discrete_is_loan_10000'].value_counts()

In [ ]:
train['discrete_rate_greater_917'].value_counts()

In [ ]:
pd.crosstab(train['discrete_is_loan_10000'], train['loan_status'])

In [6]:
import lightgbm as lgb
train_dataset = lgb.Dataset(train.drop(columns='loan_status'), train['loan_status'])
test_dataset = lgb.Dataset(test.drop(columns='loan_status'), test['loan_status'])

In [18]:
train_shaffle = train.sample(frac=1)
train_new = lgb.Dataset(train.drop(columns='loan_status'), train['loan_status'])

In [9]:
param = {'num_leaves': 31, 'objective': 'binary', 'metric':'binary_error'}
num_round = 2000

In [ ]:
model = lgb.train(param, train_new, num_boost_round=num_round, valid_sets=[train_dataset, test_dataset])

In [11]:
import io
import multiprocessing
from contextlib import redirect_stdout
from copy import deepcopy
from dataclasses import dataclass, asdict
import hyperopt.pyll
from hyperopt import fmin, tpe, hp
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import torch

import copy
cpu_count = 4
use_gpu = False
@dataclass
class LGBOpt:
    num_threads: any = hp.choice('num_threads', [cpu_count])
    num_leaves: any = hp.choice('num_leaves', [64])
    metric: any = hp.choice('metric', ['binary_error'])
    num_round: any = hp.choice('num_rounds', [1000])
    objective: any = hp.choice('objective', ['binary'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    feature_fraction: any = hp.uniform('feature_fraction', 0.5, 1.0)
    bagging_fraction: any = hp.uniform('bagging_fraction', 0.8, 1.0)
    device_type: any = hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type',
                                                                                   ['cpu'])
    boosting: any = hp.choice('boosting', ['gbdt', 'dart', 'goss'])
    extra_trees: any = hp.choice('extra_tress', [False, True])
    drop_rate: any = hp.uniform('drop_rate', 0, 0.2)
    uniform_drop: any = hp.choice('uniform_drop', [True, False])
    lambda_l1: any = hp.uniform('lambda_l1', 0, 10)  # TODO: Check range
    lambda_l2: any = hp.uniform('lambda_l2', 0, 10)  # TODO: Check range
    min_gain_to_split: any = hp.uniform('min_gain_to_split', 0, 1)  # TODO: Check range
    min_data_in_bin = hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])

    @staticmethod
    def get_common_params():
        return {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 1000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
    

In [12]:
class FitterBase(object):
    def __init__(self, label, metric, max_eval=100, opt=None):
        self.label = label
        self.metric = metric
        self.opt_params = dict()
        self.max_eval = max_eval
        self.opt = opt

    def get_loss(self, y, y_pred):
        if self.metric == 'error':
            return 1 - accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return 1 - precision_score(y, y_pred)
        elif self.metric == 'recall':
            return 1 - recall_score(y, y_pred)
        elif self.metric == 'macro_f1':
            return 1 - f1_score(y, y_pred, average='macro')
        elif self.metric == 'micro_f1':
            return 1 - f1_score(y, y_pred, average='micro')
        elif self.metric == 'auc':  # TODO: Add a warning checking if y_predict is all [0, 1], it should be probability
            return 1 - roc_auc_score(y, y_pred)
        else:
            raise Exception("Not implemented yet.")


In [13]:
class LGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: LGBOpt = None, max_eval=100):
        super(LGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = LGBOpt()
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = lgb.Dataset(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = lgb.Dataset(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [dtrain, deval]
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)

        num_round = use_params.pop('num_round')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            self.best_round = num_round
            return output

    def search(self, train_df, eval_df, use_best_eval=True):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params, use_best_eval)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
                else:
                    y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                               num_iteration=self.best_round) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest, num_iteration=self.best_round)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models

In [14]:
fitter = LGBFitter(label='loan_status')

In [15]:
params = {'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary',
                'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}

In [16]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)

In [19]:
train_pred,test_pred,acc,model = fitter.train_k_fold(kfold, train_shaffle, test, params = params)

The minimum is attained in round 1028
The minimum is attained in round 989
The minimum is attained in round 1555
The minimum is attained in round 1041
The minimum is attained in round 736


In [20]:
import numpy as np
print(params)
print(np.mean(acc))

{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.07972000000000001


In [21]:
1- accuracy_score(test['loan_status'], (test_pred > 0.5).astype(np.int64))

0.08250000000000002

In [41]:
extension_train = test[0:10000].copy()

In [ ]:
extension_train.loc[:,'loan_status'] = ((test_pred[0:10000] > 0.5).astype(np.int64)) 
extension_train

In [53]:
stacking_train = train.append(extension_train,ignore_index=True)
stacking_test = test[10000:].copy()


In [54]:
train.shape
stacking_train_shaffle.shape
test.shape
stacking_test.shape

(40000, 147)

In [ ]:
# stacking
import lightgbm as lgb

stacking_train_dataset = lgb.Dataset(stacking_train.drop(columns='loan_status'), stacking_train['loan_status'])
stacking_test_dataset = lgb.Dataset(stacking_test.drop(columns='loan_status'), stacking_test['loan_status'])

stacking_train_shaffle = stacking_train.sample(frac=1)
stacking_train_new = lgb.Dataset(stacking_train_shaffle.drop(columns='loan_status'), stacking_train_shaffle['loan_status'])

param = {'num_leaves': 31, 'objective': 'binary', 'metric':'binary_error'}
num_round = 2000

model = lgb.train(param, stacking_train_new, num_boost_round=num_round, valid_sets=[stacking_train_dataset, stacking_test_dataset])

In [56]:
params = {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 4000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
kfold = KFold(n_splits=5)
_,_,acc,_ = fitter.train_k_fold(kfold, stacking_train_shaffle, stacking_test, params = params)
print(params)
print(np.mean(acc))

The minimum is attained in round 389
The minimum is attained in round 337
The minimum is attained in round 354
The minimum is attained in round 286
The minimum is attained in round 3912
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.07023333333333334


In [ ]:
from sklearn.model_selection import KFold
import numpy as np
for learning_rate in [0.015, 0.01, 0.005, 0.001]:
  params = {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 4000, 'learning_rate': learning_rate, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
  kfold = KFold(n_splits=5)
  _,_,acc,_ = fitter.train_k_fold(kfold, train, test, params = params)
  print(params)
  print(np.mean(acc))

The minimum is attained in round 562
The minimum is attained in round 624
The minimum is attained in round 649
The minimum is attained in round 799
The minimum is attained in round 424
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.015, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.07997999999999998
The minimum is attained in round 897
The minimum is attained in round 989
The minimum is attained in round 791
The minimum is attained in round 1260
The minimum is attained in round 678
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.08011999999999997
The minimum is attained in round 1801
The minimum is attained in round 2386
The minimum is attained in round 1616
The minimum is attained in round 2611
The minimum is attained in round 1333
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 

上次作业调参的最优参数： {'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8} **0.07944**

构建衍生变量：discrete_is_loan_10000，即continuous_loan_amnt是否为10000
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.07984000000000002

构建衍生变量：discrete_rate_greater_917
通过观察贷款状态为违约的利率比正常的要高很多，贷款为正常的贷款利率分布于30%和35%的是9.17%，所以尝试增加一个衍生变量：利率是否大于9.17%
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.07972000000000001

{'num_thread': 4, 'num_leaves': 16, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.08054000000000001

调参：对叶节点数进行枚举看看哪个效果好，
The minimum is attained in round 614
The minimum is attained in round 786
The minimum is attained in round 859
The minimum is attained in round 758
The minimum is attained in round 478
{'num_thread': 4, 'num_leaves': 6, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07993999999999998**
The minimum is attained in round 498
The minimum is attained in round 637
The minimum is attained in round 485
The minimum is attained in round 681
The minimum is attained in round 365
{'num_thread': 4, 'num_leaves': 8, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08005999999999998**
The minimum is attained in round 448
The minimum is attained in round 493
The minimum is attained in round 433
The minimum is attained in round 547
The minimum is attained in round 307
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07977999999999999**
The minimum is attained in round 386
The minimum is attained in round 376
The minimum is attained in round 405
The minimum is attained in round 550
The minimum is attained in round 290
{'num_thread': 4, 'num_leaves': 16, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08054000000000001**
The minimum is attained in round 316
The minimum is attained in round 293
The minimum is attained in round 329
The minimum is attained in round 289
The minimum is attained in round 274
{'num_thread': 4, 'num_leaves': 32, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08077999999999999**
The minimum is attained in round 301
The minimum is attained in round 244
The minimum is attained in round 306
The minimum is attained in round 227
The minimum is attained in round 227
{'num_thread': 4, 'num_leaves': 64, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08186000000000002**
The minimum is attained in round 368
The minimum is attained in round 216
The minimum is attained in round 232
The minimum is attained in round 215
The minimum is attained in round 211
{'num_thread': 4, 'num_leaves': 128, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.08202000000000001**

发现收敛时进行的轮数比较少，因此想减小学习率来看模型效果
The minimum is attained in round 562
The minimum is attained in round 624
The minimum is attained in round 649
The minimum is attained in round 799
The minimum is attained in round 424
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.015, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.07997999999999998
The minimum is attained in round 897
The minimum is attained in round 989
The minimum is attained in round 791
The minimum is attained in round 1260
The minimum is attained in round 678
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.08011999999999997
The minimum is attained in round 1801
The minimum is attained in round 2386
The minimum is attained in round 1616
The minimum is attained in round 2611
The minimum is attained in round 1333
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.005, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.08002000000000001
The minimum is attained in round 4001
The minimum is attained in round 4001
The minimum is attained in round 4001
The minimum is attained in round 4001
The minimum is attained in round 4001
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.001, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.08080000000000001

构建衍生变量：discrete_rate_greater_10.99
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.07975999999999998

**stacking,**
选定参数：
{'num_thread': 4, 'num_leaves': 4, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07972000000000001**

生成新的训练集：将测试集的前10000个数据用上面增加衍生变量并调参的模型生成预测结果，放入训练集中，
生成测试集，去掉用作训练集的前10000个，即为后40000个数据

The minimum is attained in round 389
The minimum is attained in round 337
The minimum is attained in round 354
The minimum is attained in round 286
The minimum is attained in round 3912
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 4000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
**0.07023333333333334**

可观察到增加stacking之后效果显著提升
